# Load Best Model

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('Model-1.h5')

In [ ]:
import tensorflow
import tensorflow as tf
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
def make_gradcam_heatmap(img_array, model):
    last_conv_layer = model.layers[-3]
    last_conv_layer_model = tensorflow.keras.Model(model.inputs, last_conv_layer.output)

    classifier_input = tensorflow.keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    x = model.layers[-2](x)
    x = model.layers[-1](x)
    
    classifier_model = tensorflow.keras.Model(classifier_input, x)

    with tf.GradientTape() as tape:
        last_conv_layer_output = last_conv_layer_model([img_array,img_array])
        tape.watch(last_conv_layer_output)
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(last_conv_layer_output, axis=-1)

    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap


# Load the Test Data

In [ ]:
import numpy as np
x_test = np.load("drive/My Drive/Test_Data/testx.npy")
y_test = np.load("drive/My Drive/Test_Data/testy.npy")
from sklearn.utils import shuffle
x_test, y_test = shuffle(x_test, y_test, random_state=0)

# Load the Train Data

In [ ]:
import numpy as np
x_train = np.load("drive/My Drive/Train_Data/train_x.npy")
y_train = np.load("drive/My Drive/Train_Data/train_y.npy")
train_mean = np.mean(x_train)
train_std = np.std(x_train)

# Perform GradCam Visualization

In [ ]:
img = x_test[20]
img_array = (x_test[20] - train_mean) / train_std
img_array = img_array.reshape(1,512,512,3)


heatmap = make_gradcam_heatmap(img_array, model)





heatmap = np.uint8(255 * heatmap)

jet = cm.get_cmap("jet")

jet_colors = jet(np.arange(256))[:, :3]
jet_heatmap = jet_colors[heatmap]

jet_heatmap = tensorflow.keras.preprocessing.image.array_to_img(jet_heatmap)
jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
jet_heatmap = tensorflow.keras.preprocessing.image.img_to_array(jet_heatmap)

superimposed_img =  img
superimposed_img = tensorflow.keras.preprocessing.image.array_to_img(superimposed_img)


save_path = "test.jpg"
superimposed_img.save(save_path)

display(Image(save_path))